**Business Objective:**

To develop a predictive model that classifies individuals as Introverts or Extroverts based on their social behavior and activity patterns, helping businesses tailor products, services, or marketing strategies to better suit personality-driven preferences.


| Column Name                  | Description                                                                 |
|-----------------------------|-----------------------------------------------------------------------------|
| `Time_spent_Alone`          | Number of hours a person spends alone on average daily.                    |
| `Stage_fear`                | Indicates whether the person has fear of public speaking (Yes/No).         |
| `Social_event_attendance`   | Number of social events attended in a month.                               |
| `Going_outside`             | Indicates if the person willingly goes outside regularly (Yes/No).         |
| `Drained_after_socializing`| Indicates if the person feels mentally drained after social interactions.  |
| `Friends_circle_size`       | Number of close or frequent friends in the person’s circle.                |
| `Post_frequency`            | Number of posts shared on social media per week.                          |
| `Personality`               | Target variable; indicates if the person is an Introvert or Extrovert.     |
